In [6]:
import os
import yaml

os.chdir('Z:/Covid19/Covid19_OMOP/new_pipeline/')
print(os.getcwd())

import pandas as pd
from omop_etl.load import Loader
# from .omop_etl.stage import STAGE

from pydma.utils import find

Z:\Covid19\Covid19_OMOP\new_pipeline


In [14]:
omop = Loader('config.yml')
STAGE = omop.stage

In [8]:
stage_t = omop.list_tables(in_schema=['stage'])
stage_t['Count'] = stage_t.Table.apply(lambda t: omop.row_count(t, schema='stage'))

In [9]:
stage_t_dict = stage_t[['Table','Count']].to_dict()
# stage_t_dict

In [10]:
stage_t_dict['Count'][7]

82876

In [11]:
def find_key(input_dict, value):
    return next((k for k, v in input_dict.items() if v == value), None)

In [16]:
loaded = omop.config.load

result = []
for t in loaded.keys():
    if t not in ['provider','care_site','location']:
        if loaded[t]:
            for part in loaded[t]:
                stg_name = STAGE[t][part]
                idx = find_key(stage_t_dict['Table'], stg_name)
                count = stage_t_dict['Count'][idx]
                result.append([t, part, count])
        else:
            stg_name = STAGE[t]
            idx = find_key(stage_t_dict['Table'], stg_name)
            count = stage_t_dict['Count'][idx]
            result.append([t, None, count])

table_counts = pd.DataFrame(result, columns=['Table', 'Part', 'Count'])
# table_counts.head(50)

In [18]:
count_diff = table_counts.groupby('Table').sum().reset_index()
count_diff['RC_Load'] = count_diff.Table.apply(lambda t: omop.row_count(t))
count_diff['RC_DeId'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='hipaa'))
count_diff['RC_Preload'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='preload') if t not in ('death','person','visit_occurrence') else 0)
count_diff

,Table,Count,RC_Load,RC_DeId,RC_Preload
0,condition_occurrence,66991594,69174446,69163062,69174446
1,death,6060,6060,6060,0
2,drug_exposure,73517391,73076098,73061994,73424152
3,measurement,288885794,344474539,344474539,344490963
4,observation,71379644,71463493,71452858,71463493
5,person,204933,204973,204932,0
6,procedure_occurrence,26562539,26461419,26456997,26461419
7,visit_occurrence,23732205,23658041,23658041,0


In [ ]:
# t_sbs = stage_t[find(['MEASUREMENT','DRUG','CONDITION','PROCEDURE','OBSERVATION'], stage_t.Table)].Table.tolist()